In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd
import numpy as np

import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache, predict_next_token
from src.utils import env_utils

from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_KEY = "meta-llama/Meta-Llama-3-70B-Instruct"
svd_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, "SVDs",
    MODEL_KEY.split("/")[-1]
)

In [10]:
def patch_binding_ID(
    h_orig: torch.Tensor,
    h_other: torch.Tensor,
    Proj: torch.Tensor,
    rank: int = 10
):
    # The prediction will also change if the full h_other is patched in stead of h_orig
    h_orig = h_orig.to(torch.float32)
    h_other = h_other.to(torch.float32)
    Proj = Proj.to(torch.float32)

    Proj_r = Proj[:rank].to(lm.device)
    Proj_r = Proj_r.T @ Proj_r
    return (
        (torch.eye(Proj_r.shape[0]).to(lm.device) - Proj_r) @ h_orig # remove patching ID from s_orig
        + Proj_r @ h_other # add patching ID from s_other
    )

def remove_binding_ID(
    h: torch.Tensor,
    Proj: torch.Tensor,
    rank: int = 10
):
    h = h.to(torch.float32)
    Proj = Proj.to(torch.float32)

    Proj_r = Proj[:rank].to(lm.device)
    Proj_r = Proj_r.T @ Proj_r
    return (torch.eye(Proj_r.shape[0]).to(lm.device) - Proj_r) @ h


def load_Vh(type: str, layer_name: str):
    svd = np.load(os.path.join(svd_path, type, "projections", f"{layer_name}.npz"))
    Vh = torch.Tensor(svd['Vh'])
    return Vh

In [4]:
lm = load_LM(
    model_key=MODEL_KEY,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:03<00:00,  2.12s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [6]:
from src.dataset import SampleV3, DatasetV3

sample = SampleV3(
    protagonist="Nora",
    perpetrator="Jon",
    objects=["knife", "pen"],
    containers=["hat", "box"],
    event_idx=0,
    event_noticed=False
)

print(f"{sample.true_state=}")
print(f"{sample.protagonist_belief=}")

dataset = DatasetV3(samples = [sample])

sample.true_state={'hat': 'pen', 'box': 'pen'}
sample.protagonist_belief={'hat': 'knife', 'box': 'pen'}


In [11]:
from scripts.collect_binding_id_states import collect_token_latent_in_question

prompt, answer = dataset.__getitem__(
    0, 
    set_actor="perpetrator",
    # set_ans="no",
    set_container=0,
    set_obj=0
)
print(prompt)
print(answer)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Nora is a magician performing at a grand theater. Nora wants to amaze the audience with a trick involving a knife. Nora places the knife in a hat and sets it on the stage. Then Nora prepares a backup box and places a pen inside. An assistant named Jon, who thinks the trick should be different, swaps the knife in the hat with the pen while Nora is backstage. Nora didn't see Jon swapping the the contents of hat.
Question: Does Jon believe the hat contains knife?
Answer:
no


In [ ]:
h_perpetrator = collect_token_latent_in_question(
    lm = lm,
    prompt = prompt_perp,
    answer = answer_perp,
    token_of_interest= sample.perpetrator,
    detensorize=False
)

In [ ]:
free_gpu_cache()

patches = []
for layer_idx in range(13, 20):
    layer = LAYER_NAME_FORMAT.format(layer_idx)
    Vh = load_Vh("characters", layer)
    location = (layer, QUESTION_ACTOR_IDX)
    patches.append(PatchSpec(
        location=location,
        patch=patch_binding_ID(
            h_orig=h_perpetrator.states[location],
            h_other=h_protagonist.states[location],
            Proj=Vh,
            rank=3
        )
    ))

predict_next_token(
    lm=lm, 
    input=prompt_perp,
    patches=patches,
    interested_tokens=[14410]
)